In [1]:
'''
# V1
import streamlit as st
import pandas as pd
from PIL import Image
from data_processing.data_utils import carregar_dados
from visualization.plots import (
    criar_grafico,
    criar_grafico_barra_horizontal,
    criar_grafico_barras_verticais,
    calcular_calorias_mensais,
    calcular_km_mensais,
    criar_graficos_mensais_comparativos
)

def main():
    # Carregar os dados
    df = carregar_dados()

    with st.sidebar:
        st.subheader("Sérgio Laranjeira")
        logo_teste = Image.open("src/imagens/guerreiros.png")
        st.image(logo_teste)
    
    # Adicionar a seleção de meses com a opção 'Todos os Meses'
    meses = df['data'].dt.strftime('%Y-%m').unique()
    meses = sorted(list(meses))  # Garantir que os meses estejam ordenados
    meses = ['Todos os Meses'] + meses
    filtro_mes = st.sidebar.selectbox("Selecione o Mês", meses)

    # Filtrar os dados com base no mês selecionado
    if filtro_mes == 'Todos os Meses':
        df_filtrado = df
    else:
        df_filtrado = df[df['data'].dt.strftime('%Y-%m') == filtro_mes]

    # Calcular as médias e métricas
    media_calorias_com_zeros = df_filtrado['calorias'].mean()
    media_calorias_sem_zeros = df_filtrado[df_filtrado['calorias'] > 0]['calorias'].mean()
    media_km = df_filtrado['quilometragem'].mean()
    media_km_sem_zeros = df_filtrado[df_filtrado['quilometragem'] > 0]['quilometragem'].mean()
    media_calorias_por_minuto = df_filtrado['calorias_por_minuto'].mean()
    media_km_por_minuto = df_filtrado['km_por_minuto'].mean()

    # Calcular metas mensais
    calorias_mensais = calcular_calorias_mensais(df)
    quilometragem_mensal = calcular_km_mensais(df)

    # Exibir as métricas em 3 colunas na primeira e segunda linha
    st.write("### Métricas Calorias e KM")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("Média de Calorias geral", f"{media_calorias_com_zeros:.2f}")
    with col2:
        st.metric("Média de Calorias dias úteis", f"{media_calorias_sem_zeros:.2f}")
    with col3:
        st.metric("Média de Calorias por Minuto", f"{media_calorias_por_minuto:.2f}")

    col4, col5, col6 = st.columns(3)
    with col4:
        st.metric("Média km geral", f"{media_km:.2f}")
    with col5:
        st.metric("Média km útil", f"{media_km_sem_zeros:.2f}")
    with col6:
        st.metric("Média de Km por Minuto", f"{media_km_por_minuto:.2f}")

    # Exibir os gráficos de metas mensais
    st.write("### Metas Mensais")
    col7, col8 = st.columns(2)
    if filtro_mes == 'Todos os Meses':
        with col7:
            for fig in criar_graficos_mensais_comparativos(calorias_mensais, tipo='calorias'):
                st.plotly_chart(fig)
        with col8:
            for fig in criar_graficos_mensais_comparativos(quilometragem_mensal, tipo='quilometragem'):
                st.plotly_chart(fig)
    else:
        with col7:
            fig_calorias = criar_graficos_mensais_comparativos(calorias_mensais, tipo='calorias', filtro_mes=filtro_mes)
            if fig_calorias:
                st.plotly_chart(fig_calorias[0])
        with col8:
            fig_km = criar_graficos_mensais_comparativos(quilometragem_mensal, tipo='quilometragem', filtro_mes=filtro_mes)
            if fig_km:
                st.plotly_chart(fig_km[0])

    # Exibir os gráficos de barras horizontais
    st.write("### Gráficos por Dia da Semana")
    media_calorias_por_dia_semana = df_filtrado.groupby('dia_semana')['calorias_por_minuto'].mean().sort_values(ascending=False)
    media_km_por_dia_semana = df_filtrado.groupby('dia_semana')['km_por_minuto'].mean().sort_values(ascending=False)
    
    fig_calorias = criar_grafico_barra_horizontal(media_calorias_por_dia_semana, 'Média de Calorias por Minuto por Dia da Semana')
    fig_km = criar_grafico_barra_horizontal(media_km_por_dia_semana, 'Média de Km por Minuto por Dia da Semana')
    
    st.plotly_chart(fig_calorias)
    st.plotly_chart(fig_km)

    # Exibir o gráfico de linha
    st.write("### Evolução Diária de Calorias")
    fig_calorias = criar_grafico(df_filtrado, media_calorias_com_zeros, media_calorias_sem_zeros)
    st.plotly_chart(fig_calorias)

if __name__ == "__main__":
    main()
'''
'''
# V2
import streamlit as st
import pandas as pd
from PIL import Image
from data_processing.data_utils import carregar_dados
from visualization.plots import (
    criar_grafico_barra_horizontal,
    criar_grafico_barras_verticais,
    calcular_calorias_mensais,
    calcular_km_mensais,
)

def main():
    # Carregar os dados
    df = carregar_dados()

    with st.sidebar:
        st.subheader("Sérgio Laranjeira")
        logo_teste = Image.open("src/imagens/guerreiros.png")
        st.image(logo_teste)
        
        # Adicionar filtro para visão
        visao = st.radio("Selecione a Visão", ["Calorias", "KM"])
        
        # Adicionar filtro de semana
        semanas = df['data'].dt.isocalendar().week.unique()
        semanas = ['Todas as Semanas'] + sorted(list(semanas))
        filtro_semana = st.selectbox("Selecione a Semana", semanas)

    # Filtrar os dados com base na semana selecionada
    if filtro_semana != 'Todas as Semanas':
        df_filtrado = df[df['data'].dt.isocalendar().week == filtro_semana]
    else:
        df_filtrado = df

    # Calcular contagem de dias úteis e dias ausentes
    dias_uteis = df_filtrado[df_filtrado['quilometragem'] > 0].shape[0]
    dias_ausentes = df_filtrado[df_filtrado['quilometragem'] == 0].shape[0]

    # Exibir as métricas em cards
    st.write("### Métricas de Dias")
    col1, col2 = st.columns(2)
    with col1:
        st.metric("Dias Úteis", dias_uteis)
    with col2:
        st.metric("Dias Ausentes", dias_ausentes)

    # Exibir o gráfico vertical de acordo com a visão selecionada
    if visao == "Calorias":
        valores_corrente = df_filtrado['calorias'].sum()
        valores_anterior = calcular_calorias_mensais(df).iloc[-2]  # Mês anterior
        titulo = "Visão Calorias"
    else:
        valores_corrente = df_filtrado['quilometragem'].sum()
        valores_anterior = calcular_km_mensais(df).iloc[-2]  # Mês anterior
        titulo = "Visão KM"

    fig = criar_grafico_barras_verticais(valores_corrente, valores_anterior, titulo)
    st.plotly_chart(fig)

    # Gráfico horizontal por dia da semana
    if visao == "Calorias":
        media_por_dia_semana = df_filtrado.groupby('dia_semana')['calorias_por_minuto'].mean().sort_values(ascending=False)
        title = 'Média de Calorias por Minuto por Dia da Semana'
    else:
        media_por_dia_semana = df_filtrado.groupby('dia_semana')['km_por_minuto'].mean().sort_values(ascending=False)
        title = 'Média de KM por Minuto por Dia da Semana'

    fig_horizontal = criar_grafico_barra_horizontal(media_por_dia_semana, title)
    st.plotly_chart(fig_horizontal)

if __name__ == "__main__":
    main()
'''

'\n# V2\nimport streamlit as st\nimport pandas as pd\nfrom PIL import Image\nfrom data_processing.data_utils import carregar_dados\nfrom visualization.plots import (\n    criar_grafico_barra_horizontal,\n    criar_grafico_barras_verticais,\n    calcular_calorias_mensais,\n    calcular_km_mensais,\n)\n\ndef main():\n    # Carregar os dados\n    df = carregar_dados()\n\n    with st.sidebar:\n        st.subheader("Sérgio Laranjeira")\n        logo_teste = Image.open("src/imagens/guerreiros.png")\n        st.image(logo_teste)\n        \n        # Adicionar filtro para visão\n        visao = st.radio("Selecione a Visão", ["Calorias", "KM"])\n        \n        # Adicionar filtro de semana\n        semanas = df[\'data\'].dt.isocalendar().week.unique()\n        semanas = [\'Todas as Semanas\'] + sorted(list(semanas))\n        filtro_semana = st.selectbox("Selecione a Semana", semanas)\n\n    # Filtrar os dados com base na semana selecionada\n    if filtro_semana != \'Todas as Semanas\':\n     

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def criar_grafico_barras_verticais(valores_corrente, valores_anterior, titulo):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=['Mês Corrente', 'Mês Anterior'],
        y=[valores_corrente, valores_anterior],
        marker=dict(color=['#212b2c', '#536c6f']),
        text=[valores_corrente, valores_anterior],
        textposition='auto'
    ))

    fig.update_layout(
        title=titulo,
        xaxis_title='Mês',
        yaxis_title='Valores',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=50)
    )

    return fig

def calcular_calorias_mensais(df):
    return df.groupby(df['data'].dt.to_period('M'))['calorias'].sum()

def calcular_km_mensais(df):
    return df.groupby(df['data'].dt.to_period('M'))['quilometragem'].sum()

def criar_graficos_mensais_comparativos(metas_mensais, tipo='calorias', filtro_mes=None):
    meses = metas_mensais.index.tolist()
    
    if filtro_mes and filtro_mes != 'Todos os Meses':
        mes_corrente = pd.Period(filtro_mes)
        mes_anterior = meses[meses.index(mes_corrente) - 1] if meses.index(mes_corrente) > 0 else mes_corrente
        meses = [mes_corrente]

    figs = []
    for mes_corrente in meses:
        mes_anterior = meses[meses.index(mes_corrente) - 1] if meses.index(mes_corrente) > 0 else mes_corrente

        valor_corrente = metas_mensais[mes_corrente] if mes_corrente in metas_mensais.index else 0
        valor_anterior = metas_mensais[mes_anterior] if mes_anterior in metas_mensais.index else 0

        titulo = f'Meta {tipo.capitalize()} Mensal ({mes_corrente})'
        fig = criar_grafico_barras_verticais(valor_corrente, valor_anterior, titulo)
        figs.append(fig)
    
    return figs

def criar_grafico_barra_horizontal(data, title):
    fig = go.Figure(go.Bar(
        x=data.values,
        y=data.index,
        orientation='h',
        marker=dict(
            color='#212b2c',
            line=dict(color='black', width=0.9)
        )
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Média',
        yaxis_title='Dia da Semana',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        margin=dict(l=100, r=20, t=50, b=50)
    )

    return fig

def criar_grafico(df, media_calorias_com_zeros, media_calorias_sem_zeros):
    fig = go.Figure()

    # Definir as cores dos pontos com base nas condições fornecidas
    cores_pontos = ['green' if calorias > media_calorias_sem_zeros else '#212b2c' if calorias > 0 else 'red' for calorias in df['calorias']]

    fig.add_trace(go.Scatter(
        x=df['data'],
        y=df['calorias'],
        mode='lines+markers',
        line=dict(color='black'),
        marker=dict(
            color=cores_pontos,
            size=10
        ),
        text=df.apply(lambda row: f"Calorias: {row['calorias']}<br>Quilometragem: {row['quilometragem']}", axis=1),
        hoverinfo='text',
        name='Calorias'
    ))

    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_sem_zeros, media_calorias_sem_zeros],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name=f'Média de Calorias (sem zeros: {media_calorias_sem_zeros:.2f})'
    ))

    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_com_zeros, media_calorias_com_zeros],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name=f'Média de Calorias (com zeros: {media_calorias_com_zeros:.2f})'
    ))

    fig.update_xaxes(
        tickmode='array',
        tickvals=df['data'],
        ticktext=df['data'].dt.strftime('%d-%b-%a'),
        tickangle=-90,
        tickfont=dict(size=10, color='black')
    )

    fig.update_layout(
        title='Evolução Diária de Calorias',
        xaxis_title='Data',
        yaxis_title='Calorias',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=150)
    )

    return fig


In [ ]:
import plotly.graph_objects as go
'''
def criar_grafico_barras_verticais(valores_corrente, valores_anterior, titulo):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=['Mês Corrente', 'Mês Anterior'],
        y=[valores_corrente, valores_anterior],
        marker=dict(color=['#212b2c', '#536c6f']),
        text=[valores_corrente, valores_anterior],
        textposition='auto'
    ))

    fig.update_layout(
        title=titulo,
        xaxis_title='Mês',
        yaxis_title='Valores',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=50)
    )

    return fig


def criar_grafico_barra_horizontal(data, title):
    fig = go.Figure(go.Bar(
        x=data.values,
        y=data.index,
        orientation='h',
        marker=dict(
            color='#212b2c',
            line=dict(color='black', width=0.9)
        )
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Média',
        yaxis_title='Dia da Semana',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        margin=dict(l=100, r=20, t=50, b=50)
    )

    return fig

'''


def criar_grafico_barras_verticais(df_agrupado, titulo):
    fig = go.Figure(go.Bar(
        x=df_agrupado.index,
        y=df_agrupado.values,
        marker=dict(color='#212b2c'),
        text=df_agrupado.values,
        textposition='auto'
    ))

    fig.update_layout(
        title=titulo,
        xaxis_title='Dia',
        yaxis_title='Valores',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=50)
    )
    return fig

def criar_grafico_barra_horizontal(df_agrupado, titulo):
    fig = go.Figure(go.Bar(
        x=df_agrupado.values,
        y=df_agrupado.index,
        orientation='h',
        marker=dict(
            color='#212b2c',
            line=dict(color='black', width=0.9)
        )
    ))

    fig.update_layout(
        title=titulo,
        xaxis_title='Média',
        yaxis_title='Dia da Semana',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=50)
    )
    return fig


def calcular_calorias_mensais(df):
    return df.groupby(df['data'].dt.to_period('M'))['calorias'].sum()

def calcular_km_mensais(df):
    return df.groupby(df['data'].dt.to_period('M'))['quilometragem'].sum()


def criar_grafico(df, media_calorias_com_zeros, media_calorias_sem_zeros):
    fig = go.Figure()

    # Definir as cores dos pontos com base nas condições fornecidas
    cores_pontos = ['green' if calorias > media_calorias_sem_zeros else '#212b2c' if calorias > 0 else 'red' for calorias in df['calorias']]

    fig.add_trace(go.Scatter(
        x=df['data'],
        y=df['calorias'],
        mode='lines+markers',
        line=dict(color='black'),
        marker=dict(
            color=cores_pontos,
            size=10
        ),
        text=df.apply(lambda row: f"Calorias: {row['calorias']}<br>Quilometragem: {row['quilometragem']}", axis=1),
        hoverinfo='text',
        name='Calorias'
    ))

    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_sem_zeros, media_calorias_sem_zeros],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name=f'Média de Calorias (sem zeros: {media_calorias_sem_zeros:.2f})'
    ))

    fig.add_trace(go.Scatter(
        x=[df['data'].min(), df['data'].max()],
        y=[media_calorias_com_zeros, media_calorias_com_zeros],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name=f'Média de Calorias (com zeros: {media_calorias_com_zeros:.2f})'
    ))

    fig.update_xaxes(
        tickmode='array',
        tickvals=df['data'],
        ticktext=df['data'].dt.strftime('%d-%b-%a'),
        tickangle=-90,
        tickfont=dict(size=10, color='black')
    )

    fig.update_layout(
        title='Evolução Diária de Calorias',
        xaxis_title='Data',
        yaxis_title='Calorias',
        plot_bgcolor='lightgrey',
        paper_bgcolor='lightgrey',
        font=dict(color='black'),
        title_font=dict(color='black'),
        xaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        yaxis=dict(
            title_font=dict(color='black'),
            tickfont=dict(color='black')
        ),
        legend=dict(font=dict(color='black')),
        autosize=True,
        margin=dict(l=100, r=20, t=50, b=150)
    )

    return fig


In [ ]:
import streamlit as st
import pandas as pd
from PIL import Image
from data_processing.data_utils import carregar_dados
from visualization.plots import (
    criar_grafico,
    criar_grafico_barra_horizontal,
    criar_grafico_barras_verticais,
    calcular_calorias_mensais,
    calcular_km_mensais,
    # criar_graficos_mensais_comparativos
)

def main():
    # Carregar os dados
    df = carregar_dados()

    with st.sidebar:
        st.subheader("Sérgio Laranjeira")
        logo_teste = Image.open("src/imagens/guerreiros.png")
        st.image(logo_teste)

    # Filtros
    meses = df['data'].dt.strftime('%Y-%m').unique()
    meses = sorted(list(meses))
    meses = ['Todos os Meses'] + meses
    filtro_mes = st.sidebar.selectbox("Selecione o Mês", meses)

    semanas = df['data'].dt.strftime('%U-%Y').unique()
    filtro_semana = st.sidebar.selectbox("Selecione a Semana", ['Todas as Semanas'] + list(semanas))

    visao = st.sidebar.radio("Selecione a Visão", ["Visão KM", "Visão Calorias"])

    # Filtragem de dados
    if filtro_mes != 'Todos os Meses':
        df = df[df['data'].dt.strftime('%Y-%m') == filtro_mes]

    if filtro_semana != 'Todas as Semanas':
        df = df[df['data'].dt.strftime('%U-%Y') == filtro_semana]

    # Cálculo de dias úteis e ausentes
    dias_uteis = df[df['quilometragem'] > 0].shape[0]
    dias_ausentes = df[df['quilometragem'] == 0].shape[0]

    # Exibir os cards
    st.write("### Métricas Gerais")
    col1, col2, col3, col4, col5 = st.columns(5)
    with col1:
        st.metric("Média de Calorias geral", f"{df['calorias'].mean():.2f}")
    with col2:
        st.metric("Média de Calorias dias úteis", f"{df[df['calorias'] > 0]['calorias'].mean():.2f}")
    with col3:
        st.metric("Média de Calorias por Minuto", f"{df['calorias_por_minuto'].mean():.2f}")
    with col4:
        st.metric("Dias Úteis", f"{dias_uteis}")
    with col5:
        st.metric("Dias Ausentes", f"{dias_ausentes}")

    # Gráficos
    if visao == "Visão KM":
        df_agrupado = df.groupby('data')['quilometragem'].sum()
        titulo_grafico = "Média de KM por Dia"
        
    else:
        df_agrupado = df.groupby('data')['calorias'].sum()
        titulo_grafico = "Média de Calorias por Dia"

    st.write("### Gráfico de Barras Verticais")
    st.plotly_chart(criar_grafico_barras_verticais(df_agrupado, titulo_grafico))

    st.write("### Gráfico de Barras Horizontais")
    st.plotly_chart(criar_grafico_barra_horizontal(df_agrupado, titulo_grafico))

    # Manter o gráfico de linha original
    st.write("### Evolução Diária de Calorias")
    fig_calorias = criar_grafico(df, df['calorias'].mean(), df[df['calorias'] > 0]['calorias'].mean())
    st.plotly_chart(fig_calorias)

if __name__ == "__main__":
    main()
